# Polygon.io Example Notebooks
In these examples, we will leverage `polygon-api-client`. To ensure the examples run with no code changes, put your api key in the `POLYGON_API_KEY` environment variable. You can do this from python by running the following before any other cells:

```python
import os
os.environ["POLYGON_API_KEY"] = "..."  # Your api key here
```

## Example - Building a company profile widget with ipywidgets
In this example, we'll build an interactive company profile widget with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).


In [1]:
# imports

# python standard library
import os
from datetime import datetime, timedelta
from io import BytesIO

# Polygon Rest Client
from polygon import RESTClient

# Pandas
import pandas as pd

# ipywidgets
import ipywidgets as ipy

# requests
import requests

In [2]:
# Instantiate client
client = RESTClient(os.environ["POLYGON_API_KEY"])

In [3]:
# First, lets build an autocomplete widget
# instantiate a combobox
combo = ipy.Combobox()

def lookup(change):
    results = client.reference_tickers_v3(search=change["new"].upper(), limit=50).results
    combo.options = [r["ticker"] for r in results]

combo.observe(lookup, "value")
combo

Combobox(value='')

In [4]:
# Now lets build a little profile widget

# First lets grab a sample result
ticker = "AAPL"
data = client.reference_ticker_details_vx(ticker).results
data

{'ticker': 'AAPL',
 'name': 'Apple Inc.',
 'market': 'stocks',
 'locale': 'us',
 'primary_exchange': 'XNAS',
 'type': 'CS',
 'active': True,
 'currency_name': 'usd',
 'cik': '0000320193',
 'composite_figi': 'BBG000B9XRY4',
 'share_class_figi': 'BBG001S5N8V8',
 'market_cap': 2785881981445.4,
 'phone_number': '(408) 996-1010',
 'address': {'address1': 'ONE APPLE PARK WAY',
  'city': 'CUPERTINO',
  'state': 'CA',
  'postal_code': '95014'},
 'description': "Apple designs a wide variety of consumer electronic devices, including smartphones (iPhone), tablets (iPad), PCs (Mac), smartwatches (Apple Watch), AirPods, and TV boxes (Apple TV), among others. The iPhone makes up the majority of Apple's total revenue. In addition, Apple offers its customers a variety of services such as Apple Music, iCloud, Apple Care, Apple TV+, Apple Arcade, Apple Card, and Apple Pay, among others. Apple's products run internally developed software and semiconductors, and the firm is well known for its integration 

In [5]:
key = os.environ["POLYGON_API_KEY"]
quote = client.stocks_equities_snapshot_single_ticker(ticker).ticker
day = quote.day
prevDay = quote.prev_day

In [6]:
# Now lets build our data tabs

# Let's do this in a function to make our life easier down the road
def buildTabs(data, prevDay, day):
    tabs = ipy.Tab(
            children=[
                # First, our overview tab
                ipy.VBox(children=[      
                    # Company description
                    ipy.HTML(value="<p>{description}</p>".format(description=data["description"])),
                    ipy.VBox(children=[
                        ipy.HBox(children=[
                            # Primary Exchange
                            ipy.Text(
                                value=data["primary_exchange"],
                                description='Exchange:',
                                disabled=True
                            ),
                            # Sector
                            ipy.Text(
                                value=data["sic_description"].title(),
                                description='Sector:',
                                disabled=True
                            ),
                        ]),
                        ipy.HBox(children=[
                            # CIK
                            ipy.Text(
                                value=data["cik"],
                                description='CIK:',
                                disabled=True
                            ),
                            # FIGI
                            ipy.Text(
                                value=data["composite_figi"],
                                description='FIGI:',
                                disabled=True
                            ),
                        ]),
                    ]),
                ]),

                # Second, our stats tab
                ipy.VBox(children=[
                    ipy.HBox(children=[
                        ipy.FloatText(value=data["market_cap"], description="Mkt Cap:", disabled=True),
                        ipy.IntText(value=data["weighted_shares_outstanding"], description="Shares:", disabled=True)
                    ]),
                    ipy.HBox(children=[
                        ipy.FloatText(value=prevDay.close_price, description="Prev Close:", disabled=True),
                        ipy.IntText(value=prevDay.volume, description="Prev Volume:", disabled=True)
                    ]),
                ]),

                # And finally our quote tab
                ipy.VBox(children=[
                    ipy.HBox(children=[
                        ipy.FloatText(value=day.open_price, description="Open:", disabled=True),
                        ipy.FloatText(value=day.close_price, description="Last:", disabled=True),
                    ]),
                    ipy.HBox(children=[
                        ipy.FloatText(value=day.high_price, description="High:", disabled=True),
                        ipy.FloatText(value=day.low_price, description="Low:", disabled=True),
                    ]),
                ]),
            ],          
        )

    tabs.set_title(0, "Overview")
    tabs.set_title(1, "Stats")
    tabs.set_title(2, "Quote")

    return tabs

In [7]:
buildTabs(data, prevDay, day)

In [8]:
# and finally out outer container

def stockProfile(data, prevDay, day):
    return ipy.VBox(children=[
        ipy.Image(value=requests.get(data["branding"]["icon_url"]+f"?apiKey={key}").content, width=150),
        ipy.HBox(children=[
            ipy.HTML(value="<h1>{ticker}</h1>".format(ticker=data["ticker"])),
            ipy.HTML(value="<h2>&nbsp; - &nbsp; {name}</h2>".format(name=data["name"])),
        ]),
        buildTabs(data, prevDay, day),
    ])
    
stockProfile(data, prevDay, day)

In [9]:
# Now, to wire the two components together

# wire the combo box to the data handler
def handleData(_):
    ticker = combo.value
    data = client.reference_ticker_details_vx(ticker).results

    quote = client.stocks_equities_snapshot_single_ticker(ticker).ticker
    day = quote.day
    prevDay = quote.prev_day


    # build new profile widget
    accordion.children[1].children = [stockProfile(data, prevDay, day)]

    # set active tab to data
    accordion.selected_index = 1

# run data handler on ticker select
combo.on_submit(handleData)

# Render data in accordion
accordion = ipy.Accordion(children=[
    ipy.VBox(children=[
        ipy.Label("Select a ticker below and press enter"),
        combo,
    ]),
    ipy.VBox()
])

# Set titles
accordion.set_title(0, "Ticker")
accordion.set_title(1, "Data")

# and show
accordion

Accordion(children=(VBox(children=(Label(value='Select a ticker below and press enter'), Combobox(value=''))),…